In [62]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [63]:
html = requests.get("https://ny.milesplit.com/meets/563514/results")
soup = BeautifulSoup(html.content, 'html.parser')

fileList = soup.find('ul', id='resultFileList')
file_list = fileList.find_all('a')

link1_list = []
for race in file_list:
    link = race.get('href')
    link1_list.append(link)

link_list =[]
for link in link1_list:
    link2 = link.replace('formatted', 'raw')
    link_list.append(link2)

link_list

['https://ny.milesplit.com/meets/563514-nysphsaa-new-york-state-championships-2023/results/989365/raw/',
 'https://ny.milesplit.com/meets/563514-nysphsaa-new-york-state-championships-2023/results/989366/raw/',
 'https://ny.milesplit.com/meets/563514-nysphsaa-new-york-state-championships-2023/results/989367/raw/',
 'https://ny.milesplit.com/meets/563514-nysphsaa-new-york-state-championships-2023/results/989368/raw/',
 'https://ny.milesplit.com/meets/563514-nysphsaa-new-york-state-championships-2023/results/989369/raw/',
 'https://ny.milesplit.com/meets/563514-nysphsaa-new-york-state-championships-2023/results/989370/raw/',
 'https://ny.milesplit.com/meets/563514-nysphsaa-new-york-state-championships-2023/results/989371/raw/',
 'https://ny.milesplit.com/meets/563514-nysphsaa-new-york-state-championships-2023/results/989372/raw/']

In [83]:
# for link in link_list:
    
#     site = requests.get(link)
#     soupy = BeautifulSoup(site.content, 'html.parser')
#     results = soupy.find('table', class_='eventTable')

#     places = [place.text for place in results.find_all('td', class_='place')]
#     names = [name.text for name in results.find_all('td', class_='athlete').find('a')]
#     grades = [grade.text for grade in results.find_all('td', class_='grade')]
#     teams = [team.text for team in results.find_all('td', class_='team').find('a')]
#     times = [time.text for time in results.find_all('td', class_='finish')]

site = requests.get('https://ny.milesplit.com/meets/563514-nysphsaa-new-york-state-championships-2023/results/989369/raw/')
soupy = BeautifulSoup(site.content, 'html.parser')
results = soupy.find('div', id='meetResultsBody')
results_text = results.find('pre').text
justRunners = results_text.replace('-', '').replace('PLACE', '').replace('PTS', '').replace('GRADE', '').replace('SCHOOL', '').replace('TIME','').replace('PACE','').replace('NAME','').replace('  ','##').replace(' ','').replace('##',' ')
splittedRunners = justRunners.split('\r\n')

splRun_list = []
for runner in splittedRunners:
    splRun_list.append(runner)

splRun_listList = []
for runner in splRun_list:
    temp = runner.split(' ')
    splRun_listList.append(temp)

# removes empty and space strings to make all sublists to have the same amount of elements
for i in range(len(splRun_listList)):
    j=0
    while j < len(splRun_listList[i]):
        # print(i, j)
        element = splRun_listList[i][j]
        if element == '':
            splRun_listList[i].remove(element)
        elif element == ' ':
            splRun_listList[i].remove(element)
        else:
            j=j+1

# remove empty lists & title
splRun_listList = splRun_listList[1:]
splRun_listList = [notempty for notempty in splRun_listList if notempty]

#remove random 4th col number
for i in range(len(splRun_listList)):
    if splRun_listList[i][3] in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
        del splRun_listList[i][3]

splRun_list

['GIRLSCLASSA5k',
 '',
 '                         ',
 '',
 '  1  1EmilyBush       SR 1 SaratogaSprings     17:34.3 5:40',
 '  2   ZarielMacchia     JR 1 WilliamFloyd      17:44.3 5:43',
 '  3   TsadiaBercuvitz    FR 1 Ithaca          18:30.8 5:58',
 '  4  2AlyciaHart      SR 2 SaratogaSprings     18:35.1 5:59',
 '  5   AddisonVellekoop   SO 1 Guilderland       18:36.7 6:00',
 '  6   LillieBogdan     JR 2 Frontier         18:44.5 6:02',
 '  7  3AnyaBelisle      SR 3 SaratogaSprings     18:47.8 6:03',
 '  8   HaylieSmith      JR 3 RushHenrietta      18:54.9 6:06',
 '  9   EllaMasem       JR 4 Bellport         18:55.8 6:06',
 ' 10   LoganKuhn       SO 2 Bethlehem        18:59.0 6:07',
 ' 11  4MiaWickard      FR 2 Northport        19:00.3 6:07',
 ' 12   KatherineBannigan   JR 5 Bethlehem        19:01.2 6:08',
 ' 13  5AlexaBriggs      SR 4 ChurchvilleChili    19:02.9 6:08',
 ' 14   ZariaHall       SO 3 EastMeadow       19:07.1 6:10',
 ' 15   AbbyKowalczyk     JR 6 RoyCKetcham      19:09.2 6

In [84]:
places = []
names = []
grades = []
teams = []
times = []

for list in splRun_listList:
    places.append(list[0])
    names.append(list[1])
    grades.append(list[2])
    teams.append(list[3])
    times.append(list[4])

#final cleaning
i=0
for name in names:
    for char in name:
        if char.isdigit() == True:
            rep = names[i].replace(char, '')
            names[i] = rep
    i=i+1
j=0
for grade in grades:
    for char in grade:
        if char.isdigit() == True:
            rep = grades[j].replace(char, '')
            grades[j] = rep
    j=j+1

#turn to df
results_df = pd.DataFrame({'PLACE': places, 'NAME': names, 'GRADE': grades, 'TEAM': teams, 'TIME': times})

results_df

,PLACE,NAME,GRADE,TEAM,TIME
0,1,EmilyBush,SR,SaratogaSprings,17:34.3
1,2,ZarielMacchia,JR,WilliamFloyd,17:44.3
2,3,TsadiaBercuvitz,FR,Ithaca,18:30.8
3,4,AlyciaHart,SR,SaratogaSprings,18:35.1
4,5,AddisonVellekoop,SO,Guilderland,18:36.7
...,...,...,...,...,...
102,103,CamilaTorres,SR,Arlington,22:09.0
103,104,AliceWierer,SO,OrchardPark,22:18.8
104,105,CharlotteKnutsen,SR,FayettevilleManlius,22:28.1
105,106,GiannaAffatato,SO,Syosset,22:28.3
